In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np
import re
from datetime import datetime

# ***데이터 불러오기***

In [5]:
pt1 = pd.read_csv('plant1_train.csv')
pt2 = pd.read_csv('plant2_train.csv')
pt_test = pd.read_csv('plant_test.csv')

# ***필요한 함수 정의***

In [9]:
# date 변환
def date_convert(df,plant):
    date_ls = []
    for i in range(len(df)):
        date0 = re.sub('[\"]','',df[plant+'_train.mea_ddhr'].iloc[i])

        date_ls.append(datetime.strptime(date0,'%Y-%m-%d %H:%M'))
        
    return date_ls

# N시간 뒤의 데이터를 찾아서 붙여주는 함수(테스트 데이터셋 형식으로 재구축)
def after_pred_label(df,plant,n_day):

    loc1_pred, loc2_pred, loc3_pred, pred_idx = [], [], [], []
    location = 1
    for i in range(len(df)):
        crt_date = df['date'].iloc[i] # 현재 날짜 
        ft_date = df['date'].iloc[i] + timedelta(days=n_day) # 현재 날짜로부터 + n_day
        
        if i != 0 and i % 1000 == 0 :
            print(str(i) + '번 째 처리중')

        for j in range(i, len(df)):

            if ft_date == df['date'].iloc[j]: # + n_day 한 날짜와 같은지 조건문으로 탐색
                
                # 조건에 맞는 인덱스 j를 찾았다면 임의의 각 location_predict값으로 데이터를 넣음
                loc1_pred0 = df[plant+'_train.cond_loc'+str(location)].iloc[j]
                loc2_pred0 = df[plant+'_train.cond_loc'+str(location+1)].iloc[j]
                loc3_pred0 = df[plant+'_train.cond_loc'+str(location+2)].iloc[j]

                pred_idx0 = j

                loc1_pred.append(loc1_pred0)
                loc2_pred.append(loc2_pred0)
                loc3_pred.append(loc3_pred0)

                pred_idx.append(pred_idx0)
                
                

                break
            else:
                
                #조건에 맞는 날짜가 없다면 x를 반환하고 non값으로 대신 값을 넣음
                #예를들어 마지막 날은 다음 날 데이터가 없으므로 'non' 값이 들어갈 것임
                loc1_pred0 = 'x'
                loc2_pred0 = 'x'
                loc3_pred0 = 'x'
                pred_idx0 = 'x'
                
                continue
                
        if loc1_pred0 == 'x':
            loc1_pred.append('non')
            loc2_pred.append('non')
            loc3_pred.append('non')

            pred_idx.append('non')

    return loc1_pred, loc2_pred, loc3_pred, pred_idx


In [6]:
pt1['date'] = date_convert(pt1,'plant1')

In [7]:
pt2['date'] = date_convert(pt2,'plant2')

# ***Test 데이터와 형식을 맞춰주기 위한 코드(24시간 뒤)***

### ***plant1***

In [ ]:
loc1_pred, loc2_pred, loc3_pred, pred_idx = after_pred_label(pt1,'plant1',1)

pt1['after24_loc1_predict'] = loc1_pred
pt1['after24_loc2_predict'] = loc2_pred
pt1['after24_loc3_predict'] = loc3_pred
pt1['after24_idx'] = pred_idx

In [ ]:
pt1_ver2 = pt1
pt1_ver2.dropna(axis=0,inplace=True)
pt1_ver3 = pt1_ver2.replace('non',np.nan)
pt1_ver3.dropna(axis=0,inplace=True)

col_ls = pt1_ver3.columns.tolist()

loc1_col = []
loc2_col = []
loc3_col = []
for i in col_ls:
    if 'loc1' in i:
        loc1_col.append(i)
    elif 'loc2' in i:
        loc2_col.append(i)
    elif 'loc3' in i:
        loc3_col.append(i)
        
loc1_col.append('date')
loc2_col.append('date')
loc3_col.append('date')

loc1_col = list(set(loc1_col) - set(['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']))
loc2_col = list(set(loc2_col))
loc3_col = list(set(loc3_col))

loc1_col = pd.Series(loc1_col).sort_values().tolist()
loc2_col = pd.Series(loc2_col).sort_values().tolist()
loc3_col = pd.Series(loc3_col).sort_values().tolist()

pt1_loc1_df = pt1_ver3[loc1_col]
pt1_loc2_df = pt1_ver3[loc2_col]
pt1_loc3_df = pt1_ver3[loc3_col]

pt1_loc1_df = pd.concat([pt1_loc1_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)
pt1_loc2_df = pd.concat([pt1_loc2_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)
pt1_loc3_df = pd.concat([pt1_loc3_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)

pt1_loc1_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt1_loc2_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt1_loc3_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']


pt1_loc1_df['loc'] = 1
pt1_loc1_df['plant'] = 1

pt1_loc2_df['loc'] = 2
pt1_loc2_df['plant'] = 1

pt1_loc3_df['loc'] = 3
pt1_loc3_df['plant'] = 1

train_pt1 = pd.concat([pt1_loc1_df,pt1_loc2_df,pt1_loc3_df],axis=0)


### ***plant2***

In [ ]:
plt2_loc1_pred, plt2_loc2_pred, plt2_loc3_pred, plt2_pred_idx = after_pred_label(pt2,'plant2',1)

pt2['after24_loc1_predict'] = plt2_loc1_pred
pt2['after24_loc2_predict'] = plt2_loc2_pred
pt2['after24_loc3_predict'] = plt2_loc3_pred
pt2['after24_idx'] = plt2_pred_idx

In [ ]:
pt2_ver2 = pt2
pt2_ver2.dropna(axis=0,inplace=True)
pt2_ver3 = pt2_ver2.replace('non',np.nan)
pt2_ver3.dropna(axis=0,inplace=True)

col_ls = pt2_ver3.columns.tolist()

loc1_col = []
loc2_col = []
loc3_col = []
for i in col_ls:
    if 'loc1' in i:
        loc1_col.append(i)
    elif 'loc2' in i:
        loc2_col.append(i)
    elif 'loc3' in i:
        loc3_col.append(i)
        
loc1_col.append('date')
loc2_col.append('date')
loc3_col.append('date')

loc1_col = list(set(loc1_col) - set(['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']))
loc2_col = list(set(loc2_col))
loc3_col = list(set(loc3_col))

loc1_col = pd.Series(loc1_col).sort_values().tolist()
loc2_col = pd.Series(loc2_col).sort_values().tolist()
loc3_col = pd.Series(loc3_col).sort_values().tolist()

pt2_loc1_df = pt2_ver3[loc1_col]
pt2_loc2_df = pt2_ver3[loc2_col]
pt2_loc3_df = pt2_ver3[loc3_col]

pt2_loc1_df = pd.concat([pt2_loc1_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)
pt2_loc2_df = pd.concat([pt2_loc2_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)
pt2_loc3_df = pd.concat([pt2_loc3_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)

pt2_loc1_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc2_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc3_df.columns = ['after24_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc1_df['loc'] = 1
pt2_loc1_df['plant'] = 2

pt2_loc2_df['loc'] = 2
pt2_loc2_df['plant'] = 2

pt2_loc3_df['loc'] = 3
pt2_loc3_df['plant'] = 2

train_pt2 = pd.concat([pt2_loc1_df,pt2_loc2_df,pt2_loc3_df],axis=0)

# ***Test 데이터와 형식을 맞춰주기 위한 코드(48시간 뒤)***

### ***plant1***

In [ ]:
loc1_pred, loc2_pred, loc3_pred, pred_idx = after_pred_label(pt1,'plant1',2)

pt1['after48_loc1_predict'] = loc1_pred
pt1['after48_loc2_predict'] = loc2_pred
pt1['after48_loc3_predict'] = loc3_pred
pt1['after48_idx'] = pred_idx

In [ ]:
pt1_ver2 = pt1
pt1_ver2.dropna(axis=0,inplace=True)
pt1_ver3 = pt1_ver2.replace('non',np.nan)
pt1_ver3.dropna(axis=0,inplace=True)

col_ls = pt1_ver3.columns.tolist()

loc1_col = []
loc2_col = []
loc3_col = []
for i in col_ls:
    if 'loc1' in i:
        loc1_col.append(i)
    elif 'loc2' in i:
        loc2_col.append(i)
    elif 'loc3' in i:
        loc3_col.append(i)
        
loc1_col.append('date')
loc2_col.append('date')
loc3_col.append('date')

loc1_col = list(set(loc1_col) - set(['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']))
loc2_col = list(set(loc2_col))
loc3_col = list(set(loc3_col))

loc1_col = pd.Series(loc1_col).sort_values().tolist()
loc2_col = pd.Series(loc2_col).sort_values().tolist()
loc3_col = pd.Series(loc3_col).sort_values().tolist()

pt1_loc1_df = pt1_ver3[loc1_col]
pt1_loc2_df = pt1_ver3[loc2_col]
pt1_loc3_df = pt1_ver3[loc3_col]

pt1_loc1_df = pd.concat([pt1_loc1_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)
pt1_loc2_df = pd.concat([pt1_loc2_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)
pt1_loc3_df = pd.concat([pt1_loc3_df,pt1_ver3[['plant1_train.tem_out_loc1','plant1_train.hum_out_loc1']]],axis=1)

pt1_loc1_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt1_loc2_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt1_loc3_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']


pt1_loc1_df['loc'] = 1
pt1_loc1_df['plant'] = 1

pt1_loc2_df['loc'] = 2
pt1_loc2_df['plant'] = 1

pt1_loc3_df['loc'] = 3
pt1_loc3_df['plant'] = 1

train_pt1 = pd.concat([pt1_loc1_df,pt1_loc2_df,pt1_loc3_df],axis=0)


### ***plant2***

In [ ]:
plt2_loc1_pred, plt2_loc2_pred, plt2_loc3_pred, plt2_pred_idx = after_pred_label(pt2,'plant2',2)

pt2['after48_loc1_predict'] = plt2_loc1_pred
pt2['after48_loc2_predict'] = plt2_loc2_pred
pt2['after48_loc3_predict'] = plt2_loc3_pred
pt2['after48_idx'] = plt2_pred_idx

In [ ]:
pt2_ver2 = pt2
pt2_ver2.dropna(axis=0,inplace=True)
pt2_ver3 = pt2_ver2.replace('non',np.nan)
pt2_ver3.dropna(axis=0,inplace=True)


col_ls = pt2_ver3.columns.tolist()

loc1_col = []
loc2_col = []
loc3_col = []
for i in col_ls:
    if 'loc1' in i:
        loc1_col.append(i)
    elif 'loc2' in i:
        loc2_col.append(i)
    elif 'loc3' in i:
        loc3_col.append(i)
        
loc1_col.append('date')
loc2_col.append('date')
loc3_col.append('date')

loc1_col = list(set(loc1_col) - set(['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']))
loc2_col = list(set(loc2_col))
loc3_col = list(set(loc3_col))

loc1_col = pd.Series(loc1_col).sort_values().tolist()
loc2_col = pd.Series(loc2_col).sort_values().tolist()
loc3_col = pd.Series(loc3_col).sort_values().tolist()

pt2_loc1_df = pt2_ver3[loc1_col]
pt2_loc2_df = pt2_ver3[loc2_col]
pt2_loc3_df = pt2_ver3[loc3_col]

pt2_loc1_df = pd.concat([pt2_loc1_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)
pt2_loc2_df = pd.concat([pt2_loc2_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)
pt2_loc3_df = pd.concat([pt2_loc3_df,pt2_ver3[['plant2_train.tem_out_loc1','plant2_train.hum_out_loc1']]],axis=1)

pt2_loc1_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc2_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc3_df.columns = ['after48_loc_predict','date','plant_train.cond_loc',\
                       'plant_train.hum_in','plant_train.tem_coil','plant_train.tem_in',\
                      'plant_train.tem_out_loc1','plant_train.hum_out_loc1']

pt2_loc1_df['loc'] = 1
pt2_loc1_df['plant'] = 2

pt2_loc2_df['loc'] = 2
pt2_loc2_df['plant'] = 2

pt2_loc3_df['loc'] = 3
pt2_loc3_df['plant'] = 2

train_pt2 = pd.concat([pt2_loc1_df,pt2_loc2_df,pt2_loc3_df],axis=0)